![banner](./data/home-sales-shutterstock-295804091-1068x601.jpg)

# King County Home Sales
**Authors:** [Jerry Vasquez](https://www.linkedin.com/in/jerry-vasquez-832b71224/), [Paul Lindquist](https://www.linkedin.com/in/paul-lindquist/), [Vu Brown](https://www.linkedin.com/in/austin-brown-b5211384/)

## Overview
***
This is our overview

## Business Problem
***
This is our business problem

## Data
***
This is where the data is sourced from with focuses:

## Methods
***
Descriptive analysis, etc.

## Exploratory Data Analysis
***
Notes on EDA

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from math import sqrt
import time
from collections import Counter
import itertools
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
sns.set_theme(palette='magma_r')
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 100) # Allows Jupyter Notebook to expand how much data is shown.

In [ ]:
# Load DataFrame
df = pd.read_csv('./data/kc_house_data.csv')

In [ ]:
df.head(100)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.nunique(axis=0)

In [ ]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

In [ ]:
df.corr()

In [ ]:
# Features with highest correlation to price
price_corr = df.corr()['price'].map(abs).sort_values(ascending=False)
price_corr

In [ ]:
# Plot top 6 feature correlations with price
fig, axs = plt.subplots(3, 2, figsize=(15,10))
axs[0, 0].scatter(df.sqft_living, df.price)
axs[0, 0].set_title('sqft_living')
axs[0, 1].scatter(df.sqft_above, df.price)
axs[0, 1].set_title('sqft_above')
axs[1, 0].scatter(df.sqft_living15, df.price)
axs[1, 0].set_title('sqft_living15')
axs[1, 1].scatter(df.bathrooms, df.price)
axs[1, 1].set_title('bathrooms')
axs[2, 0].scatter(df.bedrooms, df.price)
axs[2, 0].set_title('bedrooms')
axs[2, 1].scatter(df.lat, df.price)
axs[2, 1].set_title('lat')
fig.tight_layout();

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(data=df, x='long', y='lat', hue='price', palette='magma_r');

## Data Cleaning, Preparation, Feature Engineering, etc.
***
Identifying and dropping duplicates

In [ ]:
# Create a function to identify duplicates
def determine_dupes(series):
    series_vcs = pd.Series(series.value_counts())
    series_dupes = [series_vcs.index[index] for index in range(len(series_vcs)) if series_vcs.values[index] > 1]
    print("Amount of unique duplicates: " + str(len(series_dupes)))
    print("Total amount of duplicates: " + str(series_vcs.values[0:len(series_dupes)].sum()))
    
    return series_vcs

In [ ]:
# Run duplicates function for 'id' series
determine_dupes(df.id)

In [ ]:
# display(df.loc[df.id == 795000620])
# display(df[df.duplicated(subset=['id'], keep=False)].head(20))
# display(df[df.duplicated(subset=['id'], keep='first')].head(10))

In [ ]:
# Drop duplicates found within 'id' series
df = df.drop_duplicates(subset=['id'], keep='last')
df.info()

In [ ]:
# df[df.duplicated(subset=['lat','long'], keep=False)].sort_values('lat')
# df = df.drop_duplicates(subset=['lat', 'long'], keep='last')

Identify and drop outliers

In [ ]:
# Examine number of bedrooms for outliers
df.bedrooms.value_counts()

In [ ]:
# df.loc[df.bedrooms == 8].sort_values('sqft_living', ascending=False).head(20)

In [ ]:
# 33 bedrooms for a 1620 sqft house is a mistake. We'll drop those values.
# 9, 10 & 11 bedrooms for houses under 5000 sqft are also a mistake. We'll drop.
df.drop(df.loc[df['bedrooms']==33].index, inplace=True)
df.drop(df.loc[df['bedrooms']==11].index, inplace=True)
df.drop(df.loc[df['bedrooms']==10].index, inplace=True)
df.drop(df.loc[df['bedrooms']==9].index, inplace=True)

df.sort_values('bedrooms', ascending=False).head(10)

In [ ]:
# df.bathrooms.value_counts()
# df.loc[df.bathrooms == 0.5]

Make features more workable by dealing with missing/bunk values and changing series from objects to integers

In [ ]:
# Replace NaN/?/missing values with 0, None or No for respective series
# Also change object series to integer via astype function
df.yr_renovated = df.yr_renovated.fillna(0)
df.yr_renovated = df.yr_renovated.astype('int64')

df.view = df.view.fillna('NONE')

df.waterfront = df.waterfront.fillna('NO')

df.loc[df.sqft_basement == '?', 'sqft_basement'] = 0.0
df.sqft_basement = df.sqft_basement.astype('float64').astype('int64')

In [ ]:
df.info()

In [ ]:
# OPTION 1: sklearn - Change 'grade' series objects to integers
# lb_make = LabelEncoder()
# df['grade'] = lb_make.fit_transform(df['grade'])
# df.grade.value_counts()
# 8:Average, 9:Good, 10:Better, 7:Low Average, 0:Very Good
# 1:Excellent, 6:Fair, 2:Luxury, 5:Low, 3:Mansion, 4:Poor

In [ ]:
# OPTION 2: pd.replace - Change 'grade' series objects to corresponding integers
df.grade = pd.to_numeric(df.grade.map(lambda x: x.split()[0]))
df['grade'].replace('3 Poor', 3, inplace=True)
df['grade'].replace('4 Low', 4, inplace=True)
df['grade'].replace('5 Fair', 5, inplace=True)
df['grade'].replace('6 Low Average', 6, inplace=True)
df['grade'].replace('7 Average', 7, inplace=True)
df['grade'].replace('8 Good', 8, inplace=True)
df['grade'].replace('9 Better', 9, inplace=True)
df['grade'].replace('10 Very Good', 10, inplace=True)
df['grade'].replace('11 Excellent', 11, inplace=True)
df['grade'].replace('12 Luxury', 12, inplace=True)
df['grade'].replace('13 Mansion', 13, inplace=True)
df.grade.value_counts()

In [ ]:
# OPTION 1: sklearn - Change 'condition' series objects to integers
# lb_make = LabelEncoder()
# df['condition'] = lb_make.fit_transform(df['condition'])
# df.condition.value_counts()
# 0:Average, 2:Good, 4:Very Good 1: Fair, 3:Poor

In [ ]:
# OPTION 2: pd.replace - Change 'condition' series objects to corresponding integers
# Integer values from https://info.kingcounty.gov/assessor/esales/Glossary.aspx
df['condition'].replace('Poor', 1, inplace=True)
df['condition'].replace('Fair', 2, inplace=True)
df['condition'].replace('Average', 3, inplace=True)
df['condition'].replace('Good', 4, inplace=True)
df['condition'].replace('Very Good', 5, inplace=True)
df.condition.value_counts()

In [ ]:
df.waterfront.value_counts()

In [ ]:
# OPTION 1: sklearn - Change 'waterfront' series objects to integers
# lb_make = LabelEncoder()
# df['waterfront'] = lb_make.fit_transform(df['waterfront'])
# df.waterfront.value_counts()
# 0:NO, 1:YES

In [ ]:
# OPTION 2: pd.cat.codes - Change 'waterfront' series objects to category
# df.waterfront = df.waterfront.astype('category')
# df.waterfront.cat.codes
# df.waterfront.value_counts()
# 0:NO, 1:YES

In [ ]:
# OPTION 3: pd.replace - Change 'waterfront' series YES/NO objects to corresponding 0/1 integers 
df['waterfront'].replace('NO', 0, inplace=True)
df['waterfront'].replace('YES', 1, inplace=True)
df.waterfront.value_counts()

In [ ]:
# OPTION 1: sklearn - Change 'view' series objects to integers
# lb_make = LabelEncoder()
# df['view'] = lb_make.fit_transform(df['view'])
# df.view.value_counts()
# 4:NONE, 0:AVERAGE, 3:GOOD, 2:FAIR, 1:EXCELLENT

In [ ]:
# OPTION 2: pd.replace - Change 'view' series objects to corresponding integers
# Integer values mirrored from 'condition' series
df['view'].replace('NONE', 0, inplace=True)
df['view'].replace('FAIR', 2, inplace=True)
df['view'].replace('AVERAGE', 3, inplace=True)
df['view'].replace('GOOD', 4, inplace=True)
df['view'].replace('EXCELLENT', 5, inplace=True)
df.view.value_counts()

In [ ]:
# Change 'date' series to datetime data type (may not be needed)
df['date'] = pd.to_datetime(df['date'])
df.info()

## Predictive Modeling
***
In this section, we create a predictive model using many correlated features. We'll normalize and scale all of the data.

In [ ]:
# Create model training and testing data
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Examine target ('price') distribution
sns.distplot(y_train, fit=stats.norm)
fig = plt.figure()
stats.probplot(y_train, plot=plt);

In [ ]:
# Run log function to normalize target data
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

In [ ]:
# Re-examine target ('price') 
sns.distplot(y_train_log, fit=stats.norm)
fig = plt.figure()
stats.probplot(y_train_log, plot=plt);

In [ ]:
# Show feature correlation of training data
train_data = pd.concat([X_train, y_train], axis=1)
corr = train_data.corr()

fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(data=corr, mask=np.triu(np.ones_like(corr, dtype=bool)),
            ax=ax,annot=True, cbar_kws={"label": "Correlation",
                                        "orientation": "horizontal",
                                        "pad": .2, "extend": "both"});

In [ ]:
# Show linear correlation with 'price' & 'sqft_living'
most_correlated_feature = 'sqft_living'

fig, ax = plt.subplots()
ax.scatter(X_train[most_correlated_feature], y_train, alpha=0.5)
ax.set_xlabel('sqft_living')
ax.set_ylabel('price')
ax.set_title('Most Correlated Feature vs. Price');

In [ ]:
# Create baseline model with DummyRegressor
baseline = DummyRegressor()
baseline.fit(X_train, y_train_log)
baseline.score(X_test, y_test_log)

In [ ]:
# Run baseline model with highested correlated feature ('sqft_living')
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, ShuffleSplit

first_model = LinearRegression()

splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

first_scores = cross_validate(estimator=first_model,
                                 X=X_train[[most_correlated_feature]],
                                 y=y_train_log, return_train_score=True,
                                 cv=splitter)

print('Train score: ', first_scores['train_score'].mean())
print('Validation score: ', first_scores['test_score'].mean())

In [ ]:
# Add additional, correlated features to X_train data
select_features = X_train[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                             'floors', 'waterfront', 'view', 'condition', 'grade',
                             'sqft_above', 'sqft_basement', 'sqft_living15',
                             'sqft_lot15']].copy()

In [ ]:
# Run 2nd model with additional, correlated features
second_model_with_ylog = LinearRegression()

second_model_scores = cross_validate(estimator=second_model_with_ylog,
                                     X=select_features, y=y_train_log,
                                     return_train_score=True, cv=splitter)

print('Second Model')
print('Train score: ', second_model_scores['train_score'].mean())
print('Validation score: ', second_model_scores['test_score'].mean())
print()
print('First Model')
print('Train score: ', first_scores['train_score'].mean())
print('Validation score: ', first_scores['test_score'].mean())

In [ ]:
# Examine OLS summary table to examine coefficients
sm.OLS(y_train_log, sm.add_constant(select_features)).fit().summary()

In [ ]:
# Remove 'sqft_basement' due to high p-value and possible multicollinearity
less_features = select_features.drop(['sqft_basement'], axis=1).copy()

In [ ]:
#Run 3rd model with 'sqft_basement' removed
third_model_with_ylog = LinearRegression()

third_model_scores = cross_validate(estimator=third_model_with_ylog,
                                     X=less_features, y=y_train_log,
                                     return_train_score=True, cv=splitter)

print('Third Model')
print('Train score: ', third_model_scores['train_score'].mean())
print('Validation score: ', third_model_scores['test_score'].mean())
print()
print('Second Model')
print('Train score: ', second_model_scores['train_score'].mean())
print('Validation score: ', second_model_scores['test_score'].mean())
print()
print('First Model')
print('Train score: ', first_scores['train_score'].mean())
print('Validation score: ', first_scores['test_score'].mean())

In [ ]:
# Use recursive feature elimination and feature selection to examine significant features
X_train_for_RFECV = StandardScaler().fit_transform(less_features)

model_for_RFECV = LinearRegression()

selector = RFECV(model_for_RFECV, cv=splitter)
selector.fit(X_train_for_RFECV, y_train_log)

print("Was the column selected?")
for index, col in enumerate(less_features.columns):
    print(f"{col}: {selector.support_[index]}")

Creating a final model with the settled-on, selected features. This is also where we'll normalize (log) and scale the remaining data (independent variables).

In [ ]:
final_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                  'waterfront', 'view', 'condition', 'grade', 'sqft_above',
                  'sqft_living15', 'sqft_lot15']

In [ ]:
# Build final model and score it

X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

final_model = LinearRegression()
final_model.fit(X_train_final, y_train_log)

final_model.score(X_test_final, y_test_log)

In [ ]:
# Check RMSE
mean_squared_error(y_test_log, final_model.predict(X_test_final), squared=False)

Now we need to log and scale independent variables (X_train, X_test) and scale target variable (y_train_log, y_test_log). Note, target already had log applied.

In [ ]:
# Examine skew of final features
X_train[final_features].hist(figsize=(12,12));

In [ ]:
# Apply log to continuous features and re-examine skew
X_train_continuous_log = pd.DataFrame([])
X_train_continuous_log['sqft_living_log'] = np.log(X_train['sqft_living'])
X_train_continuous_log['sqft_lot_log'] = np.log(X_train['sqft_lot'])
X_train_continuous_log['sqft_above_log'] = np.log(X_train['sqft_above'])
X_train_continuous_log['sqft_living15_log'] = np.log(X_train['sqft_living15'])
X_train_continuous_log['sqft_lot15_log'] = np.log(X_train['sqft_lot15'])
X_train_continuous_log.hist(figsize=(12,12));

In [ ]:
# Create a DataFrame of all train features (independent & target) so
# everything can be scaled
X_train_discreet = X_train[['bedrooms', 'bathrooms', 'floors', 'waterfront',
                           'view', 'condition', 'grade']]

X_train_cont_disc = pd.concat([X_train_continuous_log, X_train_discreet, y_train_log],
                              axis=1)

train_columns = X_train_cont_disc.columns

In [ ]:
# Scale all training features
scaler = StandardScaler()
X_train_log_scaled = scaler.fit_transform(X_train_cont_disc)

In [ ]:
# Re-separate target and independent features
X_train_full = pd.DataFrame(X_train_log_scaled, columns=train_columns)

y_train_log_scaled = X_train_full['price']
X_train_log_scaled = X_train_full.drop(columns=['price'])
X_train_log_scaled

In [ ]:
# Repeat the above process for the testing data
X_test_continuous_log = pd.DataFrame([])
X_test_continuous_log['sqft_living_log'] = np.log(X_test['sqft_living'])
X_test_continuous_log['sqft_lot_log'] = np.log(X_test['sqft_lot'])
X_test_continuous_log['sqft_above_log'] = np.log(X_test['sqft_above'])
X_test_continuous_log['sqft_living15_log'] = np.log(X_test['sqft_living15'])
X_test_continuous_log['sqft_lot15_log'] = np.log(X_test['sqft_lot15'])

X_test_discreet = X_test[['bedrooms', 'bathrooms', 'floors', 'waterfront',
                          'view', 'condition', 'grade']]
X_test_cont_disc = pd.concat([X_test_continuous_log, X_test_discreet, y_test_log],
                              axis=1)
test_columns = X_test_cont_disc.columns

scaler2 = StandardScaler()
X_test_log_scaled = scaler2.fit_transform(X_test_cont_disc)

X_test_full = pd.DataFrame(X_test_log_scaled, columns=test_columns)

y_test_log_scaled = X_test_full['price']
X_test_log_scaled = X_test_full.drop(columns=['price'])

In [ ]:
# Create, run and score final model using log and scaled data
final_model_log_scaled = LinearRegression()
final_model_log_scaled.fit(X_train_log_scaled, y_train_log_scaled)

final_model_log_scaled.score(X_test_log_scaled, y_test_log_scaled)

In [ ]:
# Find scaled RMSE
RMSE_log_scaled = mean_squared_error(y_test_log_scaled,
                   final_model_log_scaled.predict(X_test_log_scaled),
                   squared=False)
np.exp(RMSE_log_scaled)

Check Linearity

In [ ]:
preds = final_model_log_scaled.predict(X_test_log_scaled)
fig, ax = plt.subplots()

perfect_line = np.arange(y_test_log_scaled.min(), y_test_log_scaled.max())
ax.plot(perfect_line, linestyle="--", color="orange", label="Perfect Fit")
ax.scatter(y_test_log_scaled, preds, alpha=0.5)
ax.set_xlabel("Actual Price")
ax.set_ylabel("Predicted Price")
ax.legend();

Normality

In [ ]:
residuals = (y_test_log_scaled - preds)
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

Multicollinearity (Independence Assumption)

In [ ]:
vif = [variance_inflation_factor(X_train_log_scaled.values, i) for i in range(X_train_log_scaled.shape[1])]
pd.Series(vif, index=X_train_log_scaled.columns, name="Variance Inflation Factor")

Investigating Homoscedasticity

In [ ]:
fig, ax = plt.subplots()
ax.scatter(preds, residuals, alpha=0.5)
ax.plot(preds, [0 for i in range(len(X_test_log_scaled))])
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

## Results
***
Ca-ching

## Recommendations
***
Here they are:

## Overall Conclusions
***
They are: